In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# use only train_2.csv
import zipfile as zip

zip.ZipFile('/kaggle/input/web-traffic-time-series-forecasting/train_2.csv.zip').extractall()
train_2 = pd.read_csv('/kaggle/working/train_2.csv')

In [ ]:
print(train_2.shape)
train_2.head()

In [ ]:
# fill NaN to 0
train_2 = train_2.fillna(0)
train_2 = train_2.drop('Page', axis = 1)
train_2.shape

In [ ]:
dataset = train_2.iloc[train_2.shape[0] - 1, :].values

# data normalization
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset = scaler.fit_transform(np.reshape(dataset, (-1, 1)))

x_train = dataset[0:train_2.shape[1] - 1]
y_train = dataset[1:train_2.shape[1]]

x_train.shape, y_train.shape

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2)
x_train.shape

In [ ]:
from keras.models import *
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

def build_model():
    x = Input(shape = (x_train.shape[1], x_train.shape[2]))
    out = x
    
    out = LSTM(16, activation = 'relu')(out)
    out = Dense(1, activation = 'sigmoid')(out)

    model = Model(x, out, name = 'WTTSF')
    return model

model = build_model()
model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('check.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['acc'])
history = model.fit(x_train, y_train, epochs = 200, batch_size = 8, validation_data = (x_valid, y_valid), callbacks = [es, mc])

In [ ]:
# check result on graph
import matplotlib.pyplot as plt

loss = history.history['loss']
acc = history.history['acc']
val_loss = history.history['val_loss']
val_acc = history.history['val_acc']

plt.plot(loss, label= 'loss')
plt.plot(acc, label= 'acc')
plt.plot(val_loss, label= 'val_loss')
plt.plot(val_acc, label= 'val_acc')

plt.legend()
plt.show()